The goal of this workbook is to build and train the artificial neural network (ANNs). 

In [ ]:
#First run the functions workbook 
%run Functions-6-22-25.ipynb

The first ANN will be trained from Data_Set_3. The ANN will have a binary task to identify which files are simulated with a native oxide coated Si wafer substrate, and which files have a SLG substrate.  

This ANN will be refered to as ANN1

In [ ]:

def get_data_CL(path):

    os.chdir(r"XXXX") # PATH WHERE OPTICAL PROPERTIES ARE STORED
    file =  r"XXXX" # FILE FOR SLG
    SLG = pd.read_csv(file) # READ SLG
    SLG.name = 'SLG'
    E = SLG['Energy (eV)'] # TAKE ENERGY

    os.chdir(path)
    files = glob.glob(path + "/*.csv") # Loads in the files for the 1st path


    Answer_Key_Bulk_Thickness = []
    Answer_Key_EMA_Thickness = []
    Answer_Key_NTVE_Thickness = []
    Answer_Key_AngOff = []

    Answer_Key_Voidfrac =[]

    Answer_Key_Einf =[]
    Answer_Key_Amp =[]
    Answer_Key_Et = []
    Answer_Key_Br =[]
    Answer_Key_Eo =[]
    Answer_Key_Eg =[]
    Answer_Key_Ep =[]
    Answer_Key_EMA_bool = []
    Answer_Key_EMA_thickness = []
    Answer_Key_Substrate =[]
    



    file_name = []
    data =[]

    #Answer_Key_AngOff =[]

    for i in range(len(files)): # Iterate for every simulated file


            #Load in data from CSV files
            df = pd.read_csv(files[i], index_col=None)

            #Get Answer Key from CSV file
            a = str(df.iloc[0, 0])

            
            #print(a)
            #Store Answer Key as "Filename"
            file_name.append(a)
            #print(a)
            # Remove Answer Key from the data frame to avoid NAN values
            df = df.iloc[1:].reset_index(drop=True)


            # Remove data that we do not want to train the Neural Network on. This may change with other iterations
            df = df.drop(df.columns[0], axis=1)
            df.insert(0, 'Energy (eV)', E)


            #stores the data the neural network will be trained on. (N,C,S in this case)
            data.append(df)


            #Splits the answer key (a) into different sections delimited by the "_" symbol
            a = a.split('\\')[-1].split('_')
            #print(len(a))
            #print(a)
            
            #Stores the Ep parameter for this data set
            Answer_Key_Ep.append(float(a[1]))
        
            #Stores the Eg parameter for this data set
            Answer_Key_Eg.append(float(a[3]))

            #Stores the Eo parameter for this data set
            Answer_Key_Eo.append(float(a[5]))

            #Stores the Br parameter for this data set
            Answer_Key_Br.append(float(a[7]))

            #Stores the Amp parameter for this data set
            Answer_Key_Amp.append(float(a[9]))

            #Stores the Einf parameter for this data set
            Answer_Key_Einf.append(float(a[13]))  


            if len(a) == 33: 
                
                Answer_Key_EMA_bool.append(0) # EMA 0 means no EMA
                Answer_Key_EMA_thickness.append(0)
                Answer_Key_Bulk_Thickness.append(float(a[14].split('nm')[0]))
                Answer_Key_Substrate.append(1) # 1 means soda-lime glass
    
            if len(a) == 51:

                Answer_Key_EMA_bool.append(1)  # EMA 1 means there is an EMA layer
                Answer_Key_EMA_thickness.append(float(a[17].split('nm')[0]))
                Answer_Key_Bulk_Thickness.append(float(a[32].split('nm')[0]))
                Answer_Key_Substrate.append(1) # 1 means soda-lime glass

            if len(a) == 18: 

                Answer_Key_EMA_bool.append(0) # EMA 0 means no EMA
                Answer_Key_EMA_thickness.append(0)
                Answer_Key_Bulk_Thickness.append(float(a[14].split('nm')[0]))
                Answer_Key_Substrate.append(0) # 0 means Si wafer
    
            if len(a) == 36:

                Answer_Key_EMA_bool.append(1)  # EMA 1 means there is an EMA layer
                Answer_Key_EMA_thickness.append(float(a[17].split('nm')[0]))
                Answer_Key_Bulk_Thickness.append(float(a[32].split('nm')[0]))
                Answer_Key_Substrate.append(0) # 0 means Si wafer



        



    train_data = np.array(data)
    train_files = np.array(file_name)
    train_label_Ep =np.array(Answer_Key_Ep)
    train_label_Eg =np.array(Answer_Key_Eg)
    train_label_Eo =np.array(Answer_Key_Eo)
    train_label_Br =np.array(Answer_Key_Br) 
    train_label_Amp =np.array(Answer_Key_Amp)
    train_label_Einf =np.array(Answer_Key_Einf)
    train_label_BulkT = np.array(Answer_Key_Bulk_Thickness)
    train_label_EMA_bool = np.array(Answer_Key_EMA_bool)
    train_label_EMA_Thickness = np.array(Answer_Key_EMA_thickness)
    train_label_substrate = np.array(Answer_Key_Substrate)
    

    return(
        train_files, 
        train_data,
        train_label_Ep,
        train_label_Eg,
        train_label_Eo, 
        train_label_Br, 
        train_label_Amp, 
        train_label_Einf, 
        train_label_BulkT,
        train_label_EMA_bool,
        train_label_EMA_Thickness,
        train_label_substrate
         )


In [ ]:
# First load in the training from Data_Set_3. 

# Identify the location of the data. Please put path here.
path = r'C:\Users\bordo\Documents\UToledo\Research\ML\a-Si-Paper\Cody-Lorentz_Reviewer_Comments\SE_Sim\test'  ########## PLEASE PUT THE DIRECTORY OF THE TRAINING DATA OF DATA SET 3 in the XXXX space ####################

# Get the data using function "get_data_CL"
train_files, train_data, train_Ep, train_Eg, train_Eo, train_Br, train_Amp, train_Einf, train_BulkT, train_EMA_bool, train_EMAT, train_Sub_bool =  get_data_CL(path)

# Randomly Shuffle the data using function "unison_shuffled_copies"
train_files, train_data, train_Ep, train_Eg, train_Eo, train_Br, train_Amp, train_Einf, train_BulkT, train_EMA_bool, train_EMAT, train_Sub_bool  = unison_shuffled_copies( train_files, train_data, train_Ep, train_Eg, train_Eo, train_Br, train_Amp, train_Einf, train_BulkT, train_EMA_bool, train_EMAT, train_Sub_bool )

In [ ]:
idx = 0
print(train_Ep[idx])
print(train_Eg[idx])
print(train_Eo[idx])
print(train_Br[idx])
print(train_Amp[idx])
print(train_Einf[idx])
print(train_BulkT[idx])
print(train_EMA_bool[idx])
print(train_EMAT[idx])
print(train_Sub_bool[idx])

In [ ]:
train_files[idx]

Now that the data reading function is working. The data can be loaded in.

In [ ]:
# First load in the training from Data_Set_3. 

# Identify the location of the data. Please put path here.
path = r'XXXX'  ########## PLEASE PUT THE DIRECTORY OF THE TRAINING DATA OF DATA SET 3 in the XXXX space ####################

# Get the data using function "get_data_CL"
train_files, train_data, train_Ep, train_Eg, train_Eo, train_Br, train_Amp, train_Einf, train_BulkT, train_EMA_bool, train_EMAT, train_Sub_bool =  get_data_CL(path)

# Randomly Shuffle the data using function "unison_shuffled_copies"
train_files, train_data, train_Ep, train_Eg, train_Eo, train_Br, train_Amp, train_Einf, train_BulkT, train_EMA_bool, train_EMAT, train_Sub_bool  = unison_shuffled_copies( train_files, train_data, train_Ep, train_Eg, train_Eo, train_Br, train_Amp, train_Einf, train_BulkT, train_EMA_bool, train_EMAT, train_Sub_bool )

In [ ]:
# Next load in the validation data from Data_Set_3.

# Identify the location of the data. Please put path here.
path = r'XXXX'  ########## PLEASE PUT THE DIRECTORY OF THE VALIDATION DATA OF DATA SET 3 in the XXXX space ####################

# Get the data using function "get_data_CL"
val_files, val_data, val_Ep, val_Eg, val_Eo, val_Br, val_Amp, val_Einf, val_BulkT, val_EMA_bool, val_EMAT, val_Sub_bool =  get_data_CL(path)

# Randomly Shuffle the data using function "unison_shuffled_copies"
val_files, val_data, val_Ep, val_Eg, val_Eo, val_Br, val_Amp, val_Einf, val_BulkT, val_EMA_bool, val_EMAT, val_Sub_bool  = unison_shuffled_copies( val_files, val_data, val_Ep, val_Eg, val_Eo, val_Br, val_Amp, val_Einf, val_BulkT, val_EMA_bool, val_EMAT, val_Sub_bool )

In [ ]:
# Reformat the data in a way that is compatable with the ANN. 

# Identify the input data used for training the ANN
x_train = train_data # the train_data variable was collected earlier. It contains the ellipsometric spectra for each simulation 

# Identify what the output should look like. In this case, a binary identifying the substrate.
y_train = train_Sub_bool # the train_Sub_bool variable was collected earlier. It contains a binary where "0" means Si wafer and "1" means SLG

# Identify the input for the validation set 
x_val = val_data

# Identify the output for the validation set
y_val = val_Sub_bool


In [ ]:
# Now to build the model 

# Define the Input layer of the ANN.
input_shape = (697, 4) # 697 data points with the 4 columns being (Photon Energy, N, C, S)
input_NCS = tf.keras.Input(shape=input_shape)
x = tfl.Flatten()(input_NCS) # flatten the data into one array with length 2,788

# Define the hidden layers of the ANN. 
#'leaky_relu' is chosen as the activation function because it can handle negative numbers. N, C, and S can all be negative. 
x = tfl.Dense(units=512, activation='leaky_relu')(x)
x = tfl.Dense(units=256, activation='leaky_relu')(x)
x = tfl.Dense(units=128, activation='leaky_relu')(x)
x = tfl.Dense(units=64, activation='leaky_relu')(x)
x = tfl.Dense(units=32, activation='leaky_relu')(x)
x = tfl.Dense(units=16, activation='leaky_relu')(x)
x = tfl.Dense(units=8, activation='leaky_relu')(x)

# Now define the output layer. The sigmoid activation function is chosen because it is bound between [0,1]. Perfect for binary problems. 
x = tfl.Dense(units=1, activation='sigmoid')(x)

# Create the model
model = tf.keras.Model(inputs=input_NCS, outputs= x)

# Define the learning rate. The learning rate will exponetially decay with time as the model learns. This technique sometimes helps models converge.
initial_learning_rate = 1e-3 # The learning rate may need to be adjusted with training size.
decay_steps = 5000  # How many batches will be done before a decay step
decay_rate = 0.75   # How much the learning rate decays at each step. 

#This scheduler exponentially decays the learning rate based on the parameters above.
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=initial_learning_rate,
    decay_steps=decay_steps,
    decay_rate=decay_rate,
    staircase=True
)

# Compile the model. 
# The optimizer used is Adam. This is an extremly popular optimizer for training ANNs.
# The loss function is Binary Crossentropy. It is used for binary tasks.
model.compile(
    optimizer=tf.keras.optimizers.Adam(lr_schedule),
    loss =  tf.keras.losses.BinaryCrossentropy()
)

# Define early stopping. This technique calculates the loss from the validation data after every pass through the training data. 
# It will keep track of the validation loss and stop the training after the validation loss becomes stagnate. This ensures that 
# the model does not overtrain on the training data

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',      # Monitor validation loss
    patience=10,              # Number of epochs with no improvement after which training will be stopped
    verbose=1,               # Verbosity mode
    restore_best_weights=True # Restore model weights from the epoch with the best value of the monitored quantity
)

################# Optionally, an existing model can be loaded in. Below is the code to load in this model post-training.#############################
os.chdir(r"XXXX") # put path to your model here
model = load_model('ANN1.keras')

# Train the model.
history = model.fit(
    x=x_train, # Input data
    y=y_train, # Output data 
    batch_size=32, # Batch size (adjust as needed)
    epochs=50, # Number of epochs (adjust as needed)
    validation_data=(x_val, y_val), # Validation data
    verbose=1, # Verbosity mode
    callbacks=[early_stopping] # Ends training when performance on validation data stops improving
)

# Change the directory to location to save the model 
os.chdir(r"XXXX")  ########## PLEASE PUT THE DIRECTORY WHERE YOU WOULD LIKE TO SAVE THE NEW MODEL in the XXXX space ####################

##################### To save the model use the code below: ################################
model.save_weights('ANN1.weights.h5')
model.save('ANN1.keras')




The next ANN will take ellipsometric spectra as an input and make a binary decision on if a surface layer described as a Bruggeman Effective Medium Approximation (EMA) with 50% void and 50% bulk film is appropriate for the analysis. 

This ANN will be refered to as ANN2 and is also trained using Data_Set_3

In [ ]:
# Next reformat the data in a way that is compatable with the ANN. 

# Identify the input data used for training the ANN
x_train = train_data # the train_data variable was collected earlier. It contains the ellipsometric spectra for each simulation 

# Identify what the output should look like. In this case, a boolean identifying the presence of a surface layer.
y_train = train_EMA_bool # the train_EMA_bool variable was collected earlier. It contains a boolean where "0" means no surface layer and "1" means there is a surface layer 

# Identify the input for the validation set 
x_val = val_data

# Identify the output for the validation set
y_val = val_EMA_bool


In [ ]:
# Now to build the model 

# Define the Input layer of the ANN.
input_shape = (697, 4) # 697 data points with the 4 columns being (Photon Energy, N, C, S)
input_NCS = tf.keras.Input(shape=input_shape)
x = tfl.Flatten()(input_NCS) # flatten the data into one array with length 2,788

# Define the hidden layers of the ANN. 
#'leaky_relu' is chosen as the activation function because it can handle negative numbers. N, C, and S can all be negative. 
x = tfl.Dense(units=512, activation='leaky_relu')(x)
x = tfl.Dense(units=256, activation='leaky_relu')(x)
x = tfl.Dense(units=128, activation='leaky_relu')(x)
x = tfl.Dense(units=64, activation='leaky_relu')(x)
x = tfl.Dense(units=32, activation='leaky_relu')(x)
x = tfl.Dense(units=16, activation='leaky_relu')(x)
x = tfl.Dense(units=8, activation='leaky_relu')(x)

# Now define the output layer. The sigmoid activation function is chosen because it is bound between [0,1]. Perfect for binary problems. 
x = tfl.Dense(units=1, activation='sigmoid')(x)

# Create the model
model = tf.keras.Model(inputs=input_NCS, outputs= x)

# Define the learning rate. The learning rate will exponetially decay with time as the model learns. This technique sometimes helps models converge.
initial_learning_rate = 1e-6 # The learning rate may need to be adjusted with training size.
decay_steps = 2500  # How many batches will be done before a decay step
decay_rate = 0.75   # How much the learning rate decays at each step. 

#This scheduler exponentially decays the learning rate based on the parameters above.
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=initial_learning_rate,
    decay_steps=decay_steps,
    decay_rate=decay_rate,
    staircase=True
)

# Compile the model. 
# The optimizer used is Adam. This is an extremly popular optimizer for training ANNs.
# The loss function is Binary Crossentropy. It is used for binary tasks.
model.compile(
    optimizer=tf.keras.optimizers.Adam(lr_schedule),
    loss =  tf.keras.losses.BinaryCrossentropy()
)

# Define early stopping. This technique calculates the loss from the validation data after every pass through the training data. 
# It will keep track of the validation loss and stop the training after the validation loss becomes stagnate. This ensures that 
# the model does not overtrain on the training data

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',      # Monitor validation loss
    patience=10,              # Number of epochs with no improvement after which training will be stopped
    min_delta=0.0001, 
    verbose=1,               # Verbosity mode
    restore_best_weights=True # Restore model weights from the epoch with the best value of the monitored quantity
)

################# Optionally, an existing model can be loaded in. Below is the code to load in a pre-trained model .#############################
os.chdir(r"XXXX") # put path to your model here
model = load_model('ANN2.keras')


# Train the model.
history = model.fit(
    x=x_train, # Input data
    y=y_train, # Output data 
    batch_size=32, # Batch size (adjust as needed)
    epochs=200, # Number of epochs (adjust as needed)
    validation_data=(x_val, y_val), # Validation data
    verbose=1, # Verbosity mode
    callbacks=[early_stopping] # Ends training when performance on validation data stops improving
)

# Change the directory to location to save the model 
os.chdir(r"XXXX")  ########## PLEASE PUT THE DIRECTORY WHERE YOU WOULD LIKE TO SAVE THE NEW MODEL in the XXXX space ####################

##################### To save the model use the code below: ################################
model.save_weights('ANN2.weights.h5')
model.save('ANN2.keras')




ANN1 can predict the substrate, ANN2 can predict if a surface layer is needed. Now ANNs that can generate the parameter values for the Cody-Loretnz oscillator and component layer thicknesses are needed.

The achitecture was developed previously via trial and error in a direct search. The current architecture is good enough for this demonstration, but further optimization could be done by changing the hyperparameters such as: number of layers, types of hidden layers used, number of nodes in each layer, activation functions, learning rate etc. 

ANN3 will be the one trained to handel the Si / Si native oxide / bulk structure using data from Data_Set_1.
ANN4 will be the one trained to handel the SLG / SLG bulk interface / bulk / surface layer structure using data from Data_Set_2.

The different training set sizes are 10, 100, 1000, 10000, 50000, 100000. 
The size of the validation and test data are 1, 10, 1000, 5000, 10000 always being 10% the size of the training set. 

The first model will be trained ANN3 with a training set size of 10.

In [ ]:
# Get training data size = 10 

# Identify the location of the data 
path = r'XXXX'  ########## PLEASE PUT THE DIRECTORY OF THE TRAINING DATA OF DATA SET 1 with only 10 files in the XXXX space ####################

# Get the data using function "get_data_aSi_CL_E"
train_files, train_data, train_Ep, train_Eg, train_Eo, train_Br, train_Amp, train_Einf, train_BulkT, train_EMA_bool, train_EMAT, train_Sub_bool =  get_data_CL(path)

# Randomly Shuffle the data using function "unison_shuffled_copies"
train_files, train_data, train_Ep, train_Eg, train_Eo, train_Br, train_Amp, train_Einf, train_BulkT, train_EMA_bool, train_EMAT, train_Sub_bool  = unison_shuffled_copies( train_files, train_data, train_Ep, train_Eg, train_Eo, train_Br, train_Amp, train_Einf, train_BulkT, train_EMA_bool, train_EMAT, train_Sub_bool )


In [ ]:
# Get validation data size = 1 

# Identify the location of the data 
path = r'XXXX'  ########## PLEASE PUT THE DIRECTORY OF THE VALIDATION DATA OF DATA SET 1 with only 1 file in the XXXX space ####################

# Get the data using function "get_data_aSi_CL_E"
val_files, val_data, val_Ep, val_Eg, val_Eo, val_Br, val_Amp, val_Einf, val_BulkT, val_EMA_bool, val_EMAT, val_Sub_bool =  get_data_CL(path)

# Randomly Shuffle the data using function "unison_shuffled_copies"
val_files, val_data, val_Ep, val_Eg, val_Eo, val_Br, val_Amp, val_Einf, val_BulkT, val_EMA_bool, val_EMAT, val_Sub_bool  = unison_shuffled_copies( val_files, val_data, val_Ep, val_Eg, val_Eo, val_Br, val_Amp, val_Einf, val_BulkT, val_EMA_bool, val_EMAT, val_Sub_bool )

These next ANNs will be performing a regression based task instead of a binary task. When performing regression tasks, it can be benificial to standardize data such that the mean value is always 0 and the standard deviation is 1. This allows all parameters to have an equal weighting when training the model despite the parameters having different units and magnitudes (such as Amplitude, Eg, and Bulk thickness).

The function below will be used to standardize data for ANN3.

In [ ]:
def Standardize_data_ANN3(path, filename):

    #Path should be the directory you want to save summary csv files at. 
    #Filename will be used to name the summary files
   
    ####### Standardize the data ##############

    #Generate mean values from training data.
    mean_Ep = np.mean(train_Ep)
    mean_Eg = np.mean(train_Eg)
    mean_Eo = np.mean(train_Eo)
    mean_Br = np.mean(train_Br)
    mean_Amp = np.mean(train_Amp)
    mean_Einf = np.mean(train_Einf) 
    mean_BulkT = np.mean(train_BulkT)
    
    # Store the mean values in  a dictionary for future use
    Standard_Means = {
    
        'Mean_Ep' :  mean_Ep,
        'Mean_Eg' :  mean_Eg,
        'Mean_Eo' :  mean_Eo,
        'Mean_Br' :  mean_Br,
        'Mean_Amp' : mean_Amp,
        'Mean_Einf': mean_Einf,
        'Mean_BulkT': mean_BulkT 
    }
    
    # Generate standard deviations from training data.
    std_Ep = np.std(train_Ep)
    std_Eg = np.std(train_Eg)
    std_Eo = np.std(train_Eo)
    std_Br = np.std(train_Br)
    std_Amp = np.std(train_Amp)
    std_Einf = np.std(train_Einf) 
    std_BulkT = np.std(train_BulkT)
    
    # Store the standard deviations in  a dictionary for future use
    Standard_Std = {
    
        'std_Ep' :  std_Ep,
        'std_Eg' :  std_Eg,
        'std_Eo' :  std_Eo,
        'std_Br' :  std_Br,
        'std_Amp' : std_Amp,
        'std_Einf': std_Einf,
        'std_BulkT': std_BulkT,
        
    }
    
    # Generate standardized parameter values for training set 
    standardized_train_Ep = (train_Ep - mean_Ep) / std_Ep
    standardized_train_Eg = (train_Eg - mean_Eg) / std_Eg
    standardized_train_Eo = (train_Eo - mean_Eo) / std_Eo
    standardized_train_Br = (train_Br - mean_Br) / std_Br
    standardized_train_Amp = (train_Amp - mean_Amp) / std_Amp
    standardized_train_BulkT = (train_BulkT - mean_BulkT) / std_BulkT
    
    # Generate standardized parameter values for validation
    standardized_val_Ep = (val_Ep - mean_Ep) / std_Ep
    standardized_val_Eg = (val_Eg - mean_Eg) / std_Eg
    standardized_val_Eo = (val_Eo - mean_Eo) / std_Eo
    standardized_val_Br = (val_Br - mean_Br) / std_Br
    standardized_val_Amp = (val_Amp - mean_Amp) / std_Amp
    standardized_val_BulkT = (val_BulkT - mean_BulkT) / std_BulkT
    
    
    #Now the data will be formatted to be input into the ANN
    # x_train will be the input data for training
    # y_train will be the output data for training (the output data is commonly refered to as "Training Labels")
    
    x_train = train_data

    # The ANNs will have data in the format of [ Ep, Eg, Eo, Br, Amp, BulkT] where "BulkT" is the thickness of the bulk film and the other
    # parameters are the parameters associated with the Cody-lorentz oscillator. The training data is defined to follow this format. 
    y_train = np.column_stack((
        standardized_train_Ep, 
        standardized_train_Eg, 
        standardized_train_Eo, 
        standardized_train_Br, 
        standardized_train_Amp, 
        standardized_train_BulkT,
    ))
       
    # x_val will be the input data for the validation set
    # y_val will be the output data for the validation set
    x_val = val_data
    
    y_val = np.column_stack((
      
     standardized_val_Ep,
     standardized_val_Eg,
     standardized_val_Eo,
     standardized_val_Br,
     standardized_val_Amp,
     standardized_val_BulkT,
    
    ))


    # Store data in path location
    os.chdir(path) # path where data mean and standard deviations will be store
    
    ######## Store Mean ###############################
    import csv
    # Define the filename
    name =  filename + "_Mean.csv"
    
    # Get the headers from the dictionary
    headers = Standard_Means.keys()
    
    # Open a file for writing
    with open(name, mode='w', newline='') as file:
        writer = csv.writer(file)
        
        # Write the headers
        writer.writerow(headers)
        
        # Write the data rows
        row = [Standard_Means[key] for key in headers]  # Extract the values directly
        writer.writerow(row)
        
    print(f"Data saved to {filename}")
    
    ######## Store std ###############################
    
    name =   filename + "_std.csv"
    
    # Get the headers from the dictionary
    headers = Standard_Std.keys()
    
    # Open a file for writing
    with open(name, mode='w', newline='') as file:
        writer = csv.writer(file)
        
        # Write the headers
        writer.writerow(headers)
        
        # Write the data rows
        row = [Standard_Std[key] for key in headers]  # Extract the values directly
        writer.writerow(row)
        
    print(f"Data saved to {filename}")
    
    return ( x_train, y_train, x_val, y_val)
    


In [ ]:
# Use the new function:
path = r'XXXX'  ########## PLEASE PUT THE DIRECTORY WHERE YOU WOULD LIKE TO STORE THE MEAN AND STANDARD DEVIATIONS ####################
filename = 'Data_Set1_10' # 1st part of the file name

# Get Standardized data
x_train, y_train, x_val, y_val = Standardize_data_ANN3(path, filename)

Now the training and validation data is defined. The next ANN can be trained

In [ ]:
# Define input shape
input_shape = (697, 4)
input_NCS = tf.keras.Input(shape=input_shape)

# Flatten the input
x = tfl.Flatten()(input_NCS)

# Hidden Layers
x = tfl.Dense(units=2048, activation='leaky_relu')(x)
x = tfl.Dense(units=1024, activation='leaky_relu')(x)
x = tfl.Dense(units=512, activation='leaky_relu')(x)
x = tfl.Dense(units=256, activation='leaky_relu')(x)
x = tfl.Dense(units=128, activation='leaky_relu')(x)
x = tfl.Dense(units=64, activation='leaky_relu')(x)
x = tfl.Dense(units=32, activation='leaky_relu')(x)

# Output layer has 6 units. These units corrispond to [ Ep, Eg, Eo, Br, Amp, BulkT]
x = tfl.Dense(units=6, activation= None)(x)

# Create the model
model = tf.keras.Model(inputs=input_NCS, outputs= x)

# Define the learning rate. The learning rate will exponetially decay with time as the model learns. This technique sometimes helps models converge.
initial_learning_rate = 1e-3 # The learning rate may need to be adjusted with training size.
decay_steps = 2  # How many batches will be done before a decay step. 
decay_rate = 0.75   # How much the learning rate decays at each step. 

#This scheduler exponentially decays the learning rate based on the parameters above.
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=initial_learning_rate,
    decay_steps=decay_steps,
    decay_rate=decay_rate,
    staircase=True
)

# Compile the model. 
# The optimizer used is Adam. This is an extremly popular optimizer for training ANNs.
# The loss function is Mean Squared Error. Used for regression tasks
model.compile(
    optimizer=tf.keras.optimizers.Adam(lr_schedule),
    loss =  tf.keras.losses.MeanSquaredError()
)

# Define early stopping. This technique calculates the loss from the validation data after every pass through the training data. 
# It will keep track of the validation loss and stop the training after the validation loss becomes stagnate. This ensures that 
# the model does not overtrain on the training data

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',      # Monitor validation loss
    patience=10,              # Number of epochs with no improvement after which training will be stopped
    min_delta=0.0001, 
    verbose=1,               # Verbosity mode
    restore_best_weights=True # Restore model weights from the epoch with the best value of the monitored quantity
)

################# Optionally, an existing model can be loaded in. Below is the code to load in a pre-trained model .#############################
# os.chdir(r"XXXX") # put path to your model here
# model = load_model('ANN3_10.keras')



# Train the model.
history = model.fit(
    x=x_train, # Input data
    y=y_train, # Output data 
    batch_size=32, # Batch size (adjust as needed)
    epochs=200, # Number of epochs (adjust as needed)
    validation_data=(x_val, y_val), # Validation data
    verbose=1, # Verbosity mode
    callbacks=[early_stopping] # Ends training when performance on validation data stops improving
)

# Change the directory to location to save the model 
os.chdir(r"XXXX")  ########## PLEASE PUT THE DIRECTORY WHERE YOU WOULD LIKE TO SAVE THE NEW MODEL in the XXXX space ####################

##################### To save the model use the code below: ################################
model.save_weights('ANN3_10.weights.h5') #### Filename is ANN3_(# of training files)
model.save('ANN3_10.keras')


The next model will be trained with a training set size of 100

In [ ]:
# Get training data size = 100

# Identify the location of the data 
path = r'XXXX'  ########## PLEASE PUT THE DIRECTORY OF THE TRAINING DATA OF DATA SET 1 with 100 files in the XXXX space ####################

# Get the data using function "get_data_aSi_CL_E"
train_files, train_data, train_Ep, train_Eg, train_Eo, train_Br, train_Amp, train_Einf, train_BulkT, train_EMA_bool, train_EMAT, train_Sub_bool =  get_data_CL(path)

# Randomly Shuffle the data using function "unison_shuffled_copies"
train_files, train_data, train_Ep, train_Eg, train_Eo, train_Br, train_Amp, train_Einf, train_BulkT, train_EMA_bool, train_EMAT, train_Sub_bool  = unison_shuffled_copies( train_files, train_data, train_Ep, train_Eg, train_Eo, train_Br, train_Amp, train_Einf, train_BulkT, train_EMA_bool, train_EMAT, train_Sub_bool )


In [ ]:
# Get validation data size = 10 

# Identify the location of the data 
path = r'XXXX'  ########## PLEASE PUT THE DIRECTORY OF THE VALIDATION DATA OF DATA SET 1 with 10 files in the XXXX space ####################

# Get the data using function "get_data_aSi_CL_E"
val_files, val_data, val_Ep, val_Eg, val_Eo, val_Br, val_Amp, val_Einf, val_BulkT, val_EMA_bool, val_EMAT, val_Sub_bool =  get_data_CL(path)

# Randomly Shuffle the data using function "unison_shuffled_copies"
val_files, val_data, val_Ep, val_Eg, val_Eo, val_Br, val_Amp, val_Einf, val_BulkT, val_EMA_bool, val_EMAT, val_Sub_bool  = unison_shuffled_copies( val_files, val_data, val_Ep, val_Eg, val_Eo, val_Br, val_Amp, val_Einf, val_BulkT, val_EMA_bool, val_EMAT, val_Sub_bool )

In [ ]:
# Use the new function to standardize data and store the mean and standard deviation for later use. 
path = r'XXXX'  ########## PLEASE PUT THE DIRECTORY WHERE YOU WOULD LIKE TO STORE THE MEAN AND STANDARD DEVIATIONS ####################
filename = 'Data_Set1_100'

# Get Standardized data
x_train, y_train, x_val, y_val = Standardize_data_ANN3(path, filename)

In [ ]:
# Define input shape
input_shape = (697, 4)
input_NCS = tf.keras.Input(shape=input_shape)

# Flatten the input
x = tfl.Flatten()(input_NCS)

# Hidden Layers
x = tfl.Dense(units=2048, activation='leaky_relu')(x)
x = tfl.Dense(units=1024, activation='leaky_relu')(x)
x = tfl.Dense(units=512, activation='leaky_relu')(x)
x = tfl.Dense(units=256, activation='leaky_relu')(x)
x = tfl.Dense(units=128, activation='leaky_relu')(x)
x = tfl.Dense(units=64, activation='leaky_relu')(x)
x = tfl.Dense(units=32, activation='leaky_relu')(x)

# Output layer has 6 units. These units corrispond to [ Ep, Eg, Eo, Br, Amp, BulkT]
x = tfl.Dense(units=6, activation= None)(x)

# Create the model
model = tf.keras.Model(inputs=input_NCS, outputs= x)

# Define the learning rate. The learning rate will exponetially decay with time as the model learns. This technique sometimes helps models converge.
initial_learning_rate = 1e-3 # The learning rate may need to be adjusted with training size.
decay_steps = 5  # How many batches will be done before a decay step
decay_rate = 0.75   # How much the learning rate decays at each step. 

#This scheduler exponentially decays the learning rate based on the parameters above.
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=initial_learning_rate,
    decay_steps=decay_steps,
    decay_rate=decay_rate,
    staircase=True
)

# Compile the model. 
# The optimizer used is Adam. This is an extremly popular optimizer for training ANNs.
# The loss function is Mean Squared Error. Used for regression tasks
model.compile(
    optimizer=tf.keras.optimizers.Adam(lr_schedule),
    loss =  tf.keras.losses.MeanSquaredError()
)

# Define early stopping. This technique calculates the loss from the validation data after every pass through the training data. 
# It will keep track of the validation loss and stop the training after the validation loss becomes stagnate. This ensures that 
# the model does not overtrain on the training data

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',      # Monitor validation loss
    patience=10,              # Number of epochs with no improvement after which training will be stopped
    min_delta=0.0001, 
    verbose=1,               # Verbosity mode
    restore_best_weights=True # Restore model weights from the epoch with the best value of the monitored quantity
)


################# Optionally, an existing model can be loaded in. Below is the code to load in a pre-trained model .#############################
# os.chdir(r"XXXX") # put path to your model here
# model = load_model('ANN3_100.keras')


# Train the model.
history = model.fit(
    x=x_train, # Input data
    y=y_train, # Output data 
    batch_size=32, # Batch size (adjust as needed)
    epochs=200, # Number of epochs (adjust as needed)
    validation_data=(x_val, y_val), # Validation data
    verbose=1, # Verbosity mode
    callbacks=[early_stopping] # Ends training when performance on validation data stops improving
)

# Change the directory to location to save the model 
os.chdir(r"XXXX")  ########## PLEASE PUT THE DIRECTORY WHERE YOU WOULD LIKE TO SAVE THE NEW MODEL in the XXXX space ####################

##################### To save the model use the code below: ################################
model.save_weights('ANN3_100.weights.h5') #### Filename is ANN3_(# of training files)
model.save('ANN3_100.keras')


The next model will be trained with a training set size of 1000

In [ ]:
# Get training data size = 1000

# Identify the location of the data 
path = r'XXXX'  ########## PLEASE PUT THE DIRECTORY OF THE TRAINING DATA OF DATA SET 1 with only 1000 files in the XXXX space ####################

# Get the data using function "get_data_aSi_CL_E"
train_files, train_data, train_Ep, train_Eg, train_Eo, train_Br, train_Amp, train_Einf, train_BulkT, train_EMA_bool, train_EMAT, train_Sub_bool =  get_data_CL(path)

# Randomly Shuffle the data using function "unison_shuffled_copies"
train_files, train_data, train_Ep, train_Eg, train_Eo, train_Br, train_Amp, train_Einf, train_BulkT, train_EMA_bool, train_EMAT, train_Sub_bool  = unison_shuffled_copies( train_files, train_data, train_Ep, train_Eg, train_Eo, train_Br, train_Amp, train_Einf, train_BulkT, train_EMA_bool, train_EMAT, train_Sub_bool )


In [ ]:
# Get validation data size = 100 

# Identify the location of the data 
path = r'XXXX'  ########## PLEASE PUT THE DIRECTORY OF THE VALIDATION DATA OF DATA SET 1 with only 100 files in the XXXX space ####################

# Get the data using function "get_data_aSi_CL_E"
val_files, val_data, val_Ep, val_Eg, val_Eo, val_Br, val_Amp, val_Einf, val_BulkT, val_EMA_bool, val_EMAT, val_Sub_bool =  get_data_CL(path)

# Randomly Shuffle the data using function "unison_shuffled_copies"
val_files, val_data, val_Ep, val_Eg, val_Eo, val_Br, val_Amp, val_Einf, val_BulkT, val_EMA_bool, val_EMAT, val_Sub_bool  = unison_shuffled_copies( val_files, val_data, val_Ep, val_Eg, val_Eo, val_Br, val_Amp, val_Einf, val_BulkT, val_EMA_bool, val_EMAT, val_Sub_bool )

In [ ]:
# Use the new function:
path = r'XXXX'  ########## PLEASE PUT THE DIRECTORY WHERE YOU WOULD LIKE TO STORE THE MEAN AND STANDARD DEVIATIONS ####################
filename = 'Data_Set1_1000'

# Get Standardized data
x_train, y_train, x_val, y_val = Standardize_data_ANN3(path, filename)

In [ ]:
# Define input shape
input_shape = (697, 4)
input_NCS = tf.keras.Input(shape=input_shape)

# Flatten the input
x = tfl.Flatten()(input_NCS)

# Hidden Layers
x = tfl.Dense(units=2048, activation='leaky_relu')(x)
x = tfl.Dense(units=1024, activation='leaky_relu')(x)
x = tfl.Dense(units=512, activation='leaky_relu')(x)
x = tfl.Dense(units=256, activation='leaky_relu')(x)
x = tfl.Dense(units=128, activation='leaky_relu')(x)
x = tfl.Dense(units=64, activation='leaky_relu')(x)
x = tfl.Dense(units=32, activation='leaky_relu')(x)

# Output layer has 6 units. These units corrispond to [ Ep, Eg, Eo, Br, Amp, BulkT]
x = tfl.Dense(units=6, activation= None)(x)

# Create the model
model = tf.keras.Model(inputs=input_NCS, outputs= x)

# Define the learning rate. The learning rate will exponetially decay with time as the model learns. This technique sometimes helps models converge.
initial_learning_rate = 1e-3 # The learning rate may need to be adjusted with training size.
decay_steps = 50  # How many batches will be done before a decay step
decay_rate = 0.75   # How much the learning rate decays at each step. 

#This scheduler exponentially decays the learning rate based on the parameters above.
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=initial_learning_rate,
    decay_steps=decay_steps,
    decay_rate=decay_rate,
    staircase=True
)

# Compile the model. 
# The optimizer used is Adam. This is an extremly popular optimizer for training ANNs.
# The loss function is Mean Squared Error. Used for regression tasks
model.compile(
    optimizer=tf.keras.optimizers.Adam(lr_schedule),
    loss =  tf.keras.losses.MeanSquaredError()
)

# Define early stopping. This technique calculates the loss from the validation data after every pass through the training data. 
# It will keep track of the validation loss and stop the training after the validation loss becomes stagnate. This ensures that 
# the model does not overtrain on the training data

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',      # Monitor validation loss
    patience=10,              # Number of epochs with no improvement after which training will be stopped
    min_delta=0.0001, 
    verbose=1,               # Verbosity mode
    restore_best_weights=True # Restore model weights from the epoch with the best value of the monitored quantity
)


################# Optionally, an existing model can be loaded in. Below is the code to load in a pre-trained model .#############################
# os.chdir(r"XXXX") # put path to your model here
# model = load_model('ANN3_1000.keras')

# Train the model.
history = model.fit(
    x=x_train, # Input data
    y=y_train, # Output data 
    batch_size=32, # Batch size (adjust as needed)
    epochs=200, # Number of epochs (adjust as needed)
    validation_data=(x_val, y_val), # Validation data
    verbose=1, # Verbosity mode
    callbacks=[early_stopping] # Ends training when performance on validation data stops improving
)

# Change the directory to location to save the model 
os.chdir(r"XXXX") ########## PLEASE PUT THE DIRECTORY WHERE YOU WOULD LIKE TO SAVE THE NEW MODEL in the XXXX space ####################

model.save_weights('ANN3_1000.weights.h5') #### Filename is ANN3_(# of training files)
model.save('ANN3_1000.keras')


The next model will be trained with a training set size of 10000

In [ ]:
# Get training data size = 10000

# Identify the location of the data 
path = r'XXXX'  ########## PLEASE PUT THE DIRECTORY OF THE TRAINING DATA OF DATA SET 1 with only 10000 files in the XXXX space ####################

# Get the data using function "get_data_aSi_CL_E"
train_files, train_data, train_Ep, train_Eg, train_Eo, train_Br, train_Amp, train_Einf, train_BulkT, train_EMA_bool, train_EMAT, train_Sub_bool =  get_data_CL(path)

# Randomly Shuffle the data using function "unison_shuffled_copies"
train_files, train_data, train_Ep, train_Eg, train_Eo, train_Br, train_Amp, train_Einf, train_BulkT, train_EMA_bool, train_EMAT, train_Sub_bool  = unison_shuffled_copies( train_files, train_data, train_Ep, train_Eg, train_Eo, train_Br, train_Amp, train_Einf, train_BulkT, train_EMA_bool, train_EMAT, train_Sub_bool )


In [ ]:
# Get validation data size = 1000 

# Identify the location of the data 
path = r'XXXX'  ########## PLEASE PUT THE DIRECTORY OF THE VALIDATION DATA OF DATA SET 1 with only 1000 files in the XXXX space ####################

# Get the data using function "get_data_CL"
val_files, val_data, val_Ep, val_Eg, val_Eo, val_Br, val_Amp, val_Einf, val_BulkT, val_EMA_bool, val_EMAT, val_Sub_bool =  get_data_CL(path)

# Randomly Shuffle the data using function "unison_shuffled_copies"
val_files, val_data, val_Ep, val_Eg, val_Eo, val_Br, val_Amp, val_Einf, val_BulkT, val_EMA_bool, val_EMAT, val_Sub_bool  = unison_shuffled_copies( val_files, val_data, val_Ep, val_Eg, val_Eo, val_Br, val_Amp, val_Einf, val_BulkT, val_EMA_bool, val_EMAT, val_Sub_bool )

In [ ]:
# Use the new function:
path = r'XXXX'  ########## PLEASE PUT THE DIRECTORY WHERE YOU WOULD LIKE TO STORE THE MEAN AND STANDARD DEVIATIONS ####################
filename = 'Data_Set1_10000'

# Get Standardized data
x_train, y_train, x_val, y_val = Standardize_data_ANN3(path, filename)

In [ ]:
# Define input shape
input_shape = (697, 4)
input_NCS = tf.keras.Input(shape=input_shape)

# Flatten the input
x = tfl.Flatten()(input_NCS)

# Hidden Layers
x = tfl.Dense(units=2048, activation='leaky_relu')(x)
x = tfl.Dense(units=1024, activation='leaky_relu')(x)
x = tfl.Dense(units=512, activation='leaky_relu')(x)
x = tfl.Dense(units=256, activation='leaky_relu')(x)
x = tfl.Dense(units=128, activation='leaky_relu')(x)
x = tfl.Dense(units=64, activation='leaky_relu')(x)
x = tfl.Dense(units=32, activation='leaky_relu')(x)

# Output layer has 6 units. These units corrispond to [ Ep, Eg, Eo, Br, Amp, BulkT]
x = tfl.Dense(units=6, activation= None)(x)

# Create the model
model = tf.keras.Model(inputs=input_NCS, outputs= x)

# Define the learning rate. The learning rate will exponetially decay with time as the model learns. This technique sometimes helps models converge.
initial_learning_rate = 1e-6 # The learning rate may need to be adjusted with training size.
decay_steps = 500  # How many batches will be done before a decay step
decay_rate = 0.75   # How much the learning rate decays at each step. 

#This scheduler exponentially decays the learning rate based on the parameters above.
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=initial_learning_rate,
    decay_steps=decay_steps,
    decay_rate=decay_rate,
    staircase=True
)

# Compile the model. 
# The optimizer used is Adam. This is an extremly popular optimizer for training ANNs.
# The loss function is Mean Squared Error. Used for regression tasks
model.compile(
    optimizer=tf.keras.optimizers.Adam(lr_schedule),
    loss =  tf.keras.losses.MeanSquaredError()
)

# Define early stopping. This technique calculates the loss from the validation data after every pass through the training data. 
# It will keep track of the validation loss and stop the training after the validation loss becomes stagnate. This ensures that 
# the model does not overtrain on the training data

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',      # Monitor validation loss
    patience=10,              # Number of epochs with no improvement after which training will be stopped
    min_delta=0.0001, 
    verbose=1,               # Verbosity mode
    restore_best_weights=True # Restore model weights from the epoch with the best value of the monitored quantity
)

################# Optionally, an existing model can be loaded in. Below is the code to load in a pre-trained model .#############################
os.chdir(r"XXXX") # put path to your model here
model = load_model('ANN3_10000.keras')


# Train the model.
history = model.fit(
    x=x_train, # Input data
    y=y_train, # Output data 
    batch_size=32, # Batch size (adjust as needed)
    epochs=200, # Number of epochs (adjust as needed)
    validation_data=(x_val, y_val), # Validation data
    verbose=1, # Verbosity mode
    callbacks=[early_stopping] # Ends training when performance on validation data stops improving
)

# Change the directory to location to save the model 
os.chdir(r"XXXX")  ########## PLEASE PUT THE DIRECTORY WHERE YOU WOULD LIKE TO SAVE THE NEW MODEL in the XXXX space ####################

##################### To save the model use the code below: ################################
model.save_weights('ANN3_10000.weights.h5') #### Filename is ANN3_(# of training files)
model.save('ANN3_10000.keras')


The next model will be trained with a training set size of 50000

In [ ]:
# Get training data size = 50000

# Identify the location of the data 
path = r'XXXX'  ########## PLEASE PUT THE DIRECTORY OF THE TRAINING DATA OF DATA SET 1 with only 50000 files in the XXXX space ####################

# Get the data using function "get_data_aSi_CL_E"
train_files, train_data, train_Ep, train_Eg, train_Eo, train_Br, train_Amp, train_Einf, train_BulkT, train_EMA_bool, train_EMAT, train_Sub_bool =  get_data_CL(path)

# Randomly Shuffle the data using function "unison_shuffled_copies"
train_files, train_data, train_Ep, train_Eg, train_Eo, train_Br, train_Amp, train_Einf, train_BulkT, train_EMA_bool, train_EMAT, train_Sub_bool  = unison_shuffled_copies( train_files, train_data, train_Ep, train_Eg, train_Eo, train_Br, train_Amp, train_Einf, train_BulkT, train_EMA_bool, train_EMAT, train_Sub_bool )


In [ ]:
# Get validation data size = 5000 

# Identify the location of the data 
path = r'XXXX'  ########## PLEASE PUT THE DIRECTORY OF THE VALIDATION DATA OF DATA SET 1 with only 5000 files in the XXXX space ####################

# Get the data using function "get_data_aSi_CL_E"
val_files, val_data, val_Ep, val_Eg, val_Eo, val_Br, val_Amp, val_Einf, val_BulkT, val_EMA_bool, val_EMAT, val_Sub_bool =  get_data_CL(path)

# Randomly Shuffle the data using function "unison_shuffled_copies"
val_files, val_data, val_Ep, val_Eg, val_Eo, val_Br, val_Amp, val_Einf, val_BulkT, val_EMA_bool, val_EMAT, val_Sub_bool  = unison_shuffled_copies( val_files, val_data, val_Ep, val_Eg, val_Eo, val_Br, val_Amp, val_Einf, val_BulkT, val_EMA_bool, val_EMAT, val_Sub_bool )

In [ ]:
# Use the new function:
path = r'XXXX'  ########## PLEASE PUT THE DIRECTORY WHERE YOU WOULD LIKE TO STORE THE MEAN AND STANDARD DEVIATIONS ####################
filename = 'Data_Set1_50000'

# Get Standardized data
x_train, y_train, x_val, y_val = Standardize_data_ANN3(path, filename)

In [ ]:
# Define input shape
input_shape = (697, 4)
input_NCS = tf.keras.Input(shape=input_shape)

# Flatten the input
x = tfl.Flatten()(input_NCS)

# Hidden Layers
x = tfl.Dense(units=2048, activation='leaky_relu')(x)
x = tfl.Dense(units=1024, activation='leaky_relu')(x)
x = tfl.Dense(units=512, activation='leaky_relu')(x)
x = tfl.Dense(units=256, activation='leaky_relu')(x)
x = tfl.Dense(units=128, activation='leaky_relu')(x)
x = tfl.Dense(units=64, activation='leaky_relu')(x)
x = tfl.Dense(units=32, activation='leaky_relu')(x)

# Output layer has 6 units. These units corrispond to [ Ep, Eg, Eo, Br, Amp, BulkT]
x = tfl.Dense(units=6, activation= None)(x)

# Create the model
model = tf.keras.Model(inputs=input_NCS, outputs= x)

# Define the learning rate. The learning rate will exponetially decay with time as the model learns. This technique sometimes helps models converge.
initial_learning_rate = 1e-6 # The learning rate may need to be adjusted with training size.
decay_steps = 2500  # How many batches will be done before a decay step
decay_rate = 0.75   # How much the learning rate decays at each step. 

#This scheduler exponentially decays the learning rate based on the parameters above.
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=initial_learning_rate,
    decay_steps=decay_steps,
    decay_rate=decay_rate,
    staircase=True
)

# Compile the model. 
# The optimizer used is Adam. This is an extremly popular optimizer for training ANNs.
# The loss function is Mean Squared Error. Used for regression tasks
model.compile(
    optimizer=tf.keras.optimizers.Adam(lr_schedule),
    loss =  tf.keras.losses.MeanSquaredError()
)

# Define early stopping. This technique calculates the loss from the validation data after every pass through the training data. 
# It will keep track of the validation loss and stop the training after the validation loss becomes stagnate. This ensures that 
# the model does not overtrain on the training data

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',      # Monitor validation loss
    patience=10,              # Number of epochs with no improvement after which training will be stopped
    min_delta=0.0001, 
    verbose=1,               # Verbosity mode
    restore_best_weights=True # Restore model weights from the epoch with the best value of the monitored quantity
)

################# Optionally, an existing model can be loaded in. Below is the code to load in a pre-trained model .#############################
#os.chdir(r"XXXX") # put path to your model here
#model = load_model('ANN3_50000.keras')




# Train the model.
history = model.fit(
    x=x_train, # Input data
    y=y_train, # Output data 
    batch_size=32, # Batch size (adjust as needed)
    epochs=200, # Number of epochs (adjust as needed)
    validation_data=(x_val, y_val), # Validation data
    verbose=1, # Verbosity mode
    callbacks=[early_stopping] # Ends training when performance on validation data stops improving
)

# Change the directory to location to save the model 
os.chdir(r"XXXX")  ########## PLEASE PUT THE DIRECTORY WHERE YOU WOULD LIKE TO SAVE THE NEW MODEL in the XXXX space ####################

##################### To save the model use the code below: ################################
model.save_weights('ANN3_50000.weights.h5') #### Filename is ANN3_(# of training files)
model.save('ANN3_50000.keras')


The final model will be trained with a training set size of 100000

In [ ]:
# Get training data size = 100000

# Identify the location of the data 
path = r'XXXX'  ########## PLEASE PUT THE DIRECTORY OF THE TRAINING DATA OF DATA SET 1 with only 100000 files in the XXXX space ####################

# Get the data using function "get_data_aSi_CL_E"
train_files, train_data, train_Ep, train_Eg, train_Eo, train_Br, train_Amp, train_Einf, train_BulkT, train_EMA_bool, train_EMAT, train_Sub_bool =  get_data_CL(path)

# Randomly Shuffle the data using function "unison_shuffled_copies"
train_files, train_data, train_Ep, train_Eg, train_Eo, train_Br, train_Amp, train_Einf, train_BulkT, train_EMA_bool, train_EMAT, train_Sub_bool  = unison_shuffled_copies( train_files, train_data, train_Ep, train_Eg, train_Eo, train_Br, train_Amp, train_Einf, train_BulkT, train_EMA_bool, train_EMAT, train_Sub_bool )


In [ ]:
# Get validation data size = 10000 

# Identify the location of the data 
path = r'XXXX'  ########## PLEASE PUT THE DIRECTORY OF THE VALIDATION DATA OF DATA SET 1 with only 10000 files in the XXXX space ####################

# Get the data using function "get_data_aSi_CL_E"
val_files, val_data, val_Ep, val_Eg, val_Eo, val_Br, val_Amp, val_Einf, val_BulkT, val_EMA_bool, val_EMAT, val_Sub_bool =  get_data_CL(path)

# Randomly Shuffle the data using function "unison_shuffled_copies"
val_files, val_data, val_Ep, val_Eg, val_Eo, val_Br, val_Amp, val_Einf, val_BulkT, val_EMA_bool, val_EMAT, val_Sub_bool  = unison_shuffled_copies( val_files, val_data, val_Ep, val_Eg, val_Eo, val_Br, val_Amp, val_Einf, val_BulkT, val_EMA_bool, val_EMAT, val_Sub_bool )

In [ ]:
# Use the new function:
path = r'XXXX'  ########## PLEASE PUT THE DIRECTORY WHERE YOU WOULD LIKE TO STORE THE MEAN AND STANDARD DEVIATIONS ####################
filename = 'Data_Set1_100000'

# Get Standardized data
x_train, y_train, x_val, y_val = Standardize_data_ANN3(path, filename)

In [ ]:
# Define input shape
input_shape = (697, 4)
input_NCS = tf.keras.Input(shape=input_shape)

# Flatten the input
x = tfl.Flatten()(input_NCS)

# Hidden Layers
x = tfl.Dense(units=2048, activation='leaky_relu')(x)
x = tfl.Dense(units=1024, activation='leaky_relu')(x)
x = tfl.Dense(units=512, activation='leaky_relu')(x)
x = tfl.Dense(units=256, activation='leaky_relu')(x)
x = tfl.Dense(units=128, activation='leaky_relu')(x)
x = tfl.Dense(units=64, activation='leaky_relu')(x)
x = tfl.Dense(units=32, activation='leaky_relu')(x)

# Output layer has 6 units. These units corrispond to [ Ep, Eg, Eo, Br, Amp, BulkT]
x = tfl.Dense(units=6, activation= None)(x)

# Create the model
model = tf.keras.Model(inputs=input_NCS, outputs= x)

# Define the learning rate. The learning rate will exponetially decay with time as the model learns. This technique sometimes helps models converge.
initial_learning_rate = 1e-6 # The learning rate may need to be adjusted with training size.
decay_steps = 5000  # How many batches will be done before a decay step
decay_rate = 0.75   # How much the learning rate decays at each step. 

#This scheduler exponentially decays the learning rate based on the parameters above.
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=initial_learning_rate,
    decay_steps=decay_steps,
    decay_rate=decay_rate,
    staircase=True
)

# Compile the model. 
# The optimizer used is Adam. This is an extremly popular optimizer for training ANNs.
# The loss function is Mean Squared Error. Used for regression tasks
model.compile(
    optimizer=tf.keras.optimizers.Adam(lr_schedule),
    loss =  tf.keras.losses.MeanSquaredError()
)

# Define early stopping. This technique calculates the loss from the validation data after every pass through the training data. 
# It will keep track of the validation loss and stop the training after the validation loss becomes stagnate. This ensures that 
# the model does not overtrain on the training data

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',      # Monitor validation loss
    patience=10,              # Number of epochs with no improvement after which training will be stopped
    min_delta=0.0001, 
    verbose=1,               # Verbosity mode
    restore_best_weights=True # Restore model weights from the epoch with the best value of the monitored quantity
)

################# Optionally, an existing model can be loaded in. Below is the code to load in a pre-trained model .#############################
os.chdir(r"XXXX") # put path to your model here
model = load_model('ANN3_100000.keras')


# Train the model.
history = model.fit(
    x=x_train, # Input data
    y=y_train, # Output data 
    batch_size=32, # Batch size (adjust as needed)
    epochs=200, # Number of epochs (adjust as needed)
    validation_data=(x_val, y_val), # Validation data
    verbose=1, # Verbosity mode
    callbacks=[early_stopping] # Ends training when performance on validation data stops improving
)

# Change the directory to location to save the model 
os.chdir(r"XXXX")  ########## PLEASE PUT THE DIRECTORY WHERE YOU WOULD LIKE TO SAVE THE NEW MODEL in the XXXX space ####################

##################### To save the model use the code below: ################################
model.save_weights('ANN3_100000.weights.h5') #### Filename is ANN3_(# of training files)
model.save('ANN3_100000.keras')


Now moving on to ANN4 training

In [ ]:
# Get training data size = 10

# Identify the location of the data 
path = r'XXXX'  ########## PLEASE PUT THE DIRECTORY OF THE TRAINING DATA OF DATA SET 2 with 10 files in the XXXX space ####################

# Get the data using function "get_data_aSi_CL_E"
train_files, train_data, train_Ep, train_Eg, train_Eo, train_Br, train_Amp, train_Einf, train_BulkT, train_EMA_bool, train_EMAT, train_Sub_bool =  get_data_CL(path)

# Randomly Shuffle the data using function "unison_shuffled_copies"
train_files, train_data, train_Ep, train_Eg, train_Eo, train_Br, train_Amp, train_Einf, train_BulkT, train_EMA_bool, train_EMAT, train_Sub_bool  = unison_shuffled_copies( train_files, train_data, train_Ep, train_Eg, train_Eo, train_Br, train_Amp, train_Einf, train_BulkT, train_EMA_bool, train_EMAT, train_Sub_bool )


In [ ]:
# Get validation data size = 1

# Identify the location of the data 
path = r'XXXX'  ########## PLEASE PUT THE DIRECTORY OF THE VALIDATION DATA OF DATA SET 2 with 1 file in the XXXX space ####################

# Get the data using function "get_data_aSi_CL_E"
val_files, val_data, val_Ep, val_Eg, val_Eo, val_Br, val_Amp, val_Einf, val_BulkT, val_EMA_bool, val_EMAT, val_Sub_bool =  get_data_CL(path)

# Randomly Shuffle the data using function "unison_shuffled_copies"
val_files, val_data, val_Ep, val_Eg, val_Eo, val_Br, val_Amp, val_Einf, val_BulkT, val_EMA_bool, val_EMAT, val_Sub_bool  = unison_shuffled_copies( val_files, val_data, val_Ep, val_Eg, val_Eo, val_Br, val_Amp, val_Einf, val_BulkT, val_EMA_bool, val_EMAT, val_Sub_bool )

These next ANNs will be performing a regression based task instead of a binary task. When performing regression tasks, it can be benificial to standardize data such that the mean value is always 0 and the standard deviation is 1. This allows all parameters to have an equal weighting when training the model despite the parameters having different units and magnitudes (such as Amplitude, Eg, and Bulk thickness).

The function below will be used to standardize data for ANN4.

In [ ]:
# A new function to standardize the data is defined to handel the additional EMA term describing the surface layer/
# This function is nearly identical otherwise

def Standardize_data_ANN4(path, filename):
   
    ####### Standardize the data ##############

    #Generate mean values from training data.
    mean_Ep = np.mean(train_Ep)
    mean_Eg = np.mean(train_Eg)
    mean_Eo = np.mean(train_Eo)
    mean_Br = np.mean(train_Br)
    mean_Amp = np.mean(train_Amp)
    mean_Einf = np.mean(train_Einf) 
    mean_BulkT = np.mean(train_BulkT)
    mean_EMAT = np.mean(train_EMAT)
    
    # Store the mean values in  a dictionary for future use
    Standard_Means = {
    
        'Mean_Ep' :  mean_Ep,
        'Mean_Eg' :  mean_Eg,
        'Mean_Eo' :  mean_Eo,
        'Mean_Br' :  mean_Br,
        'Mean_Amp' : mean_Amp,
        'Mean_Einf': mean_Einf,
        'Mean_BulkT': mean_BulkT,
        'Mean_EMAT': mean_EMAT,
        
    }
    
    # Generate standard deviations from training data.
    std_Ep = np.std(train_Ep)
    std_Eg = np.std(train_Eg)
    std_Eo = np.std(train_Eo)
    std_Br = np.std(train_Br)
    std_Amp = np.std(train_Amp)
    std_Einf = np.std(train_Einf) 
    std_BulkT = np.std(train_BulkT)
    std_EMAT = np.std(train_EMAT)
    
    
    # Store the standard deviations in  a dictionary for future use
    Standard_Std = {
    
        'std_Ep' :  std_Ep,
        'std_Eg' :  std_Eg,
        'std_Eo' :  std_Eo,
        'std_Br' :  std_Br,
        'std_Amp' : std_Amp,
        'std_Einf': std_Einf,
        'std_BulkT': std_BulkT,
        'std_EMAT': std_EMAT,
        
    }
    
    # Generate standardized parameter values for training set 
    standardized_train_Ep = (train_Ep - mean_Ep) / std_Ep
    standardized_train_Eg = (train_Eg - mean_Eg) / std_Eg
    standardized_train_Eo = (train_Eo - mean_Eo) / std_Eo
    standardized_train_Br = (train_Br - mean_Br) / std_Br
    standardized_train_Amp = (train_Amp - mean_Amp) / std_Amp
    standardized_train_BulkT = (train_BulkT - mean_BulkT) / std_BulkT
    standardized_train_EMAT = (train_EMAT - mean_EMAT) / std_EMAT
    
    # Generate standardized parameter values for validation
    standardized_val_Ep = (val_Ep - mean_Ep) / std_Ep
    standardized_val_Eg = (val_Eg - mean_Eg) / std_Eg
    standardized_val_Eo = (val_Eo - mean_Eo) / std_Eo
    standardized_val_Br = (val_Br - mean_Br) / std_Br
    standardized_val_Amp = (val_Amp - mean_Amp) / std_Amp
    standardized_val_BulkT = (val_BulkT - mean_BulkT) / std_BulkT
    standardized_val_EMAT = (val_EMAT - mean_EMAT) / std_EMAT
    
    
    #Now the data will be formatted to be input into the ANN
    # x_train will be the input data for training
    # y_train will be the output data for training (the output data is commonly refered to as "Training Labels")
    
    x_train = train_data

    # The ANNs will have data in the format of [ Ep, Eg, Eo, Br, Amp, BulkT, EMAT] where "BulkT" is the thickness of the bulk film and 
    # "EMAT" is the thickness of the surface layer described by Bruggeman Effective Medium Approximation. The other
    # parameters are the parameters associated with the Cody-lorentz oscillator. The training data is defined to follow this format. 
    y_train = np.column_stack((
        standardized_train_Ep, 
        standardized_train_Eg, 
        standardized_train_Eo, 
        standardized_train_Br, 
        standardized_train_Amp, 
        standardized_train_BulkT,
        standardized_train_EMAT,
    ))
       
    # x_val will be the input data for the validation set
    # y_val will be the output data for the validation set
    x_val = val_data
    
    y_val = np.column_stack((
      
     standardized_val_Ep,
     standardized_val_Eg,
     standardized_val_Eo,
     standardized_val_Br,
     standardized_val_Amp,
     standardized_val_BulkT,
     standardized_val_EMAT,
    
    ))


    # Store data in path location
    os.chdir(path)
    
    ######## Store Mean ###############################
    import csv
    # Define the filename
    name =  filename + "_Mean.csv"
    
    # Get the headers from the dictionary
    headers = Standard_Means.keys()
    
    # Open a file for writing
    with open(name, mode='w', newline='') as file:
        writer = csv.writer(file)
        
        # Write the headers
        writer.writerow(headers)
        
        # Write the data rows
        row = [Standard_Means[key] for key in headers]  # Extract the values directly
        writer.writerow(row)
        
    print(f"Data saved to {filename}")
    
    ######## Store std ###############################
    
    name =   filename + "_std.csv"
    
    # Get the headers from the dictionary
    headers = Standard_Std.keys()
    
    # Open a file for writing
    with open(name, mode='w', newline='') as file:
        writer = csv.writer(file)
        
        # Write the headers
        writer.writerow(headers)
        
        # Write the data rows
        row = [Standard_Std[key] for key in headers]  # Extract the values directly
        writer.writerow(row)
        
    print(f"Data saved to {filename}")
    
    return ( x_train, y_train, x_val, y_val)
    


In [ ]:
# Use the new function:
path = r'XXXX'  ########## PLEASE PUT THE DIRECTORY WHERE YOU WOULD LIKE TO STORE THE MEAN AND STANDARD DEVIATIONS ####################
filename = 'Data_Set2_10'

# Get Standardized data
x_train, y_train, x_val, y_val = Standardize_data_ANN4(path, filename)

In [ ]:
# Define input shape
input_shape = (697, 4)
input_NCS = tf.keras.Input(shape=input_shape)

# Flatten the input
x = tfl.Flatten()(input_NCS)

# Hidden Layers
x = tfl.Dense(units=2048, activation='leaky_relu')(x)
x = tfl.Dense(units=1024, activation='leaky_relu')(x)
x = tfl.Dense(units=512, activation='leaky_relu')(x)
x = tfl.Dense(units=256, activation='leaky_relu')(x)
x = tfl.Dense(units=128, activation='leaky_relu')(x)
x = tfl.Dense(units=64, activation='leaky_relu')(x)
x = tfl.Dense(units=32, activation='leaky_relu')(x)

# Output layer has 7 units. These units corrispond to [ Ep, Eg, Eo, Br, Amp, BulkT, EMAT]
x = tfl.Dense(units=7, activation= None)(x)

# Create the model
model = tf.keras.Model(inputs=input_NCS, outputs= x)

# Define the learning rate. The learning rate will exponetially decay with time as the model learns. This technique sometimes helps models converge.
initial_learning_rate = 1e-3 # The learning rate may need to be adjusted with training size.
decay_steps = 2  # How many batches will be done before a decay step
decay_rate = 0.75   # How much the learning rate decays at each step. 

#This scheduler exponentially decays the learning rate based on the parameters above.
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=initial_learning_rate,
    decay_steps=decay_steps,
    decay_rate=decay_rate,
    staircase=True
)

# Compile the model. 
# The optimizer used is Adam. This is an extremly popular optimizer for training ANNs.
# The loss function is Mean Squared Error. Used for regression tasks
model.compile(
    optimizer=tf.keras.optimizers.Adam(lr_schedule),
    loss =  tf.keras.losses.MeanSquaredError()
)

# Define early stopping. This technique calculates the loss from the validation data after every pass through the training data. 
# It will keep track of the validation loss and stop the training after the validation loss becomes stagnate. This ensures that 
# the model does not overtrain on the training data

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',      # Monitor validation loss
    patience=10,              # Number of epochs with no improvement after which training will be stopped
    min_delta=0.0001, 
    verbose=1,               # Verbosity mode
    restore_best_weights=True # Restore model weights from the epoch with the best value of the monitored quantity
)

################# Optionally, an existing model can be loaded in. Below is the code to load in a pre-trained model .#############################
#os.chdir(r"XXXX") # put path to your model here
#model = load_model('ANN4_10.keras')



# Train the model.
history = model.fit(
    x=x_train, # Input data
    y=y_train, # Output data 
    batch_size=32, # Batch size (adjust as needed)
    epochs=200, # Number of epochs (adjust as needed)
    validation_data=(x_val, y_val), # Validation data
    verbose=1, # Verbosity mode
    callbacks=[early_stopping] # Ends training when performance on validation data stops improving
)

# Change the directory to location to save the model 
os.chdir(r"XXXX")  ########## PLEASE PUT THE DIRECTORY WHERE YOU WOULD LIKE TO SAVE THE NEW MODEL in the XXXX space ####################

##################### To save the model use the code below: ################################
model.save_weights('ANN4_10.weights.h5') #### Filename is ANN4_(# of training files)
model.save('ANN4_10.keras')


The next model will be trained with a training set size of 100

In [ ]:
# Get training data size = 100

# Identify the location of the data 
path = r'XXXX'  ########## PLEASE PUT THE DIRECTORY OF THE TRAINING DATA OF DATA SET 2 with 100 files in the XXXX space ####################

# Get the data using function "get_data_aSi_CL_E"
train_files, train_data, train_Ep, train_Eg, train_Eo, train_Br, train_Amp, train_Einf, train_BulkT, train_EMA_bool, train_EMAT, train_Sub_bool =  get_data_CL(path)

# Randomly Shuffle the data using function "unison_shuffled_copies"
train_files, train_data, train_Ep, train_Eg, train_Eo, train_Br, train_Amp, train_Einf, train_BulkT, train_EMA_bool, train_EMAT, train_Sub_bool  = unison_shuffled_copies( train_files, train_data, train_Ep, train_Eg, train_Eo, train_Br, train_Amp, train_Einf, train_BulkT, train_EMA_bool, train_EMAT, train_Sub_bool )


In [ ]:
# Get validation data size = 10

# Identify the location of the data 
path = r'XXXX'  ########## PLEASE PUT THE DIRECTORY OF THE VALIDATION DATA OF DATA SET 2 with 10 files in the XXXX space ####################

# Get the data using function "get_data_aSi_CL_E"
val_files, val_data, val_Ep, val_Eg, val_Eo, val_Br, val_Amp, val_Einf, val_BulkT, val_EMA_bool, val_EMAT, val_Sub_bool =  get_data_CL(path)

# Randomly Shuffle the data using function "unison_shuffled_copies"
val_files, val_data, val_Ep, val_Eg, val_Eo, val_Br, val_Amp, val_Einf, val_BulkT, val_EMA_bool, val_EMAT, val_Sub_bool  = unison_shuffled_copies( val_files, val_data, val_Ep, val_Eg, val_Eo, val_Br, val_Amp, val_Einf, val_BulkT, val_EMA_bool, val_EMAT, val_Sub_bool )

In [ ]:
# Use the new function:
path = r'XXXX'  ########## PLEASE PUT THE DIRECTORY WHERE YOU WOULD LIKE TO STORE THE MEAN AND STANDARD DEVIATIONS ####################
filename = 'Data_Set2_100'

# Get Standardized data
x_train, y_train, x_val, y_val = Standardize_data_ANN4(path, filename)

In [ ]:
# Define input shape
input_shape = (697, 4)
input_NCS = tf.keras.Input(shape=input_shape)

# Flatten the input
x = tfl.Flatten()(input_NCS)

# Hidden Layers
x = tfl.Dense(units=2048, activation='leaky_relu')(x)
x = tfl.Dense(units=1024, activation='leaky_relu')(x)
x = tfl.Dense(units=512, activation='leaky_relu')(x)
x = tfl.Dense(units=256, activation='leaky_relu')(x)
x = tfl.Dense(units=128, activation='leaky_relu')(x)
x = tfl.Dense(units=64, activation='leaky_relu')(x)
x = tfl.Dense(units=32, activation='leaky_relu')(x)

# Output layer has 7 units. These units corrispond to [ Ep, Eg, Eo, Br, Amp, BulkT, EMAT]
x = tfl.Dense(units=7, activation= None)(x)

# Create the model
model = tf.keras.Model(inputs=input_NCS, outputs= x)

# Define the learning rate. The learning rate will exponetially decay with time as the model learns. This technique sometimes helps models converge.
initial_learning_rate = 1e-3 # The learning rate may need to be adjusted with training size.
decay_steps = 5  # How many batches will be done before a decay step
decay_rate = 0.75   # How much the learning rate decays at each step. 

#This scheduler exponentially decays the learning rate based on the parameters above.
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=initial_learning_rate,
    decay_steps=decay_steps,
    decay_rate=decay_rate,
    staircase=True
)

# Compile the model. 
# The optimizer used is Adam. This is an extremly popular optimizer for training ANNs.
# The loss function is Mean Squared Error. Used for regression tasks
model.compile(
    optimizer=tf.keras.optimizers.Adam(lr_schedule),
    loss =  tf.keras.losses.MeanSquaredError()
)

# Define early stopping. This technique calculates the loss from the validation data after every pass through the training data. 
# It will keep track of the validation loss and stop the training after the validation loss becomes stagnate. This ensures that 
# the model does not overtrain on the training data

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',      # Monitor validation loss
    patience=10,              # Number of epochs with no improvement after which training will be stopped
    min_delta=0.0001, 
    verbose=1,               # Verbosity mode
    restore_best_weights=True # Restore model weights from the epoch with the best value of the monitored quantity
)


################# Optionally, an existing model can be loaded in. Below is the code to load in a pre-trained model .#############################
# os.chdir(r"XXXX") # put path to your model here
# model = load_model('ANN4_100.keras')

# Train the model.
history = model.fit(
    x=x_train, # Input data
    y=y_train, # Output data 
    batch_size=32, # Batch size (adjust as needed)
    epochs=200, # Number of epochs (adjust as needed)
    validation_data=(x_val, y_val), # Validation data
    verbose=1, # Verbosity mode
    callbacks=[early_stopping] # Ends training when performance on validation data stops improving
)

# Change the directory to location to save the model 
os.chdir(r"XXXX")  ########## PLEASE PUT THE DIRECTORY WHERE YOU WOULD LIKE TO SAVE THE NEW MODEL in the XXXX space ####################

##################### To save the model use the code below: ################################
model.save_weights('ANN4_100.weights.h5') #### Filename is ANN4_(# of training files)
model.save('ANN4_100.keras')


The next model will be trained with a training set size of 1000

In [ ]:
# Get training data size = 1000

# Identify the location of the data 
path = r'XXXX'  ########## PLEASE PUT THE DIRECTORY OF THE TRAINING DATA OF DATA SET 2 with 1000 files in the XXXX space ####################

# Get the data using function "get_data_aSi_CL_E"
train_files, train_data, train_Ep, train_Eg, train_Eo, train_Br, train_Amp, train_Einf, train_BulkT, train_EMA_bool, train_EMAT, train_Sub_bool =  get_data_CL(path)

# Randomly Shuffle the data using function "unison_shuffled_copies"
train_files, train_data, train_Ep, train_Eg, train_Eo, train_Br, train_Amp, train_Einf, train_BulkT, train_EMA_bool, train_EMAT, train_Sub_bool  = unison_shuffled_copies( train_files, train_data, train_Ep, train_Eg, train_Eo, train_Br, train_Amp, train_Einf, train_BulkT, train_EMA_bool, train_EMAT, train_Sub_bool )


In [ ]:
# Get validation data size = 100

# Identify the location of the data 
path = r'XXXX'  ########## PLEASE PUT THE DIRECTORY OF THE VALIDATION DATA OF DATA SET 2 with 100 files in the XXXX space ####################

# Get the data using function "get_data_aSi_CL_E"
val_files, val_data, val_Ep, val_Eg, val_Eo, val_Br, val_Amp, val_Einf, val_BulkT, val_EMA_bool, val_EMAT, val_Sub_bool =  get_data_CL(path)

# Randomly Shuffle the data using function "unison_shuffled_copies"
val_files, val_data, val_Ep, val_Eg, val_Eo, val_Br, val_Amp, val_Einf, val_BulkT, val_EMA_bool, val_EMAT, val_Sub_bool  = unison_shuffled_copies( val_files, val_data, val_Ep, val_Eg, val_Eo, val_Br, val_Amp, val_Einf, val_BulkT, val_EMA_bool, val_EMAT, val_Sub_bool )

In [ ]:
# Use the new function:
path = r'XXXX'  ########## PLEASE PUT THE DIRECTORY WHERE YOU WOULD LIKE TO STORE THE MEAN AND STANDARD DEVIATIONS ####################
filename = 'Data_Set2_1000'

# Get Standardized data
x_train, y_train, x_val, y_val = Standardize_data_ANN4(path, filename)

In [ ]:
# Define input shape
input_shape = (697, 4)
input_NCS = tf.keras.Input(shape=input_shape)

# Flatten the input
x = tfl.Flatten()(input_NCS)

# Hidden Layers
x = tfl.Dense(units=2048, activation='leaky_relu')(x)
x = tfl.Dense(units=1024, activation='leaky_relu')(x)
x = tfl.Dense(units=512, activation='leaky_relu')(x)
x = tfl.Dense(units=256, activation='leaky_relu')(x)
x = tfl.Dense(units=128, activation='leaky_relu')(x)
x = tfl.Dense(units=64, activation='leaky_relu')(x)
x = tfl.Dense(units=32, activation='leaky_relu')(x)

# Output layer has 7 units. These units corrispond to [ Ep, Eg, Eo, Br, Amp, BulkT, EMAT]
x = tfl.Dense(units=7, activation= None)(x)

# Create the model
model = tf.keras.Model(inputs=input_NCS, outputs= x)

# Define the learning rate. The learning rate will exponetially decay with time as the model learns. This technique sometimes helps models converge.
initial_learning_rate = 1e-3 # The learning rate may need to be adjusted with training size.
decay_steps = 50  # How many batches will be done before a decay step
decay_rate = 0.75   # How much the learning rate decays at each step. 

#This scheduler exponentially decays the learning rate based on the parameters above.
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=initial_learning_rate,
    decay_steps=decay_steps,
    decay_rate=decay_rate,
    staircase=True
)

# Compile the model. 
# The optimizer used is Adam. This is an extremly popular optimizer for training ANNs.
# The loss function is Mean Squared Error. Used for regression tasks
model.compile(
    optimizer=tf.keras.optimizers.Adam(lr_schedule),
    loss =  tf.keras.losses.MeanSquaredError()
)

# Define early stopping. This technique calculates the loss from the validation data after every pass through the training data. 
# It will keep track of the validation loss and stop the training after the validation loss becomes stagnate. This ensures that 
# the model does not overtrain on the training data

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',      # Monitor validation loss
    patience=10,              # Number of epochs with no improvement after which training will be stopped
    min_delta=0.0001, 
    verbose=1,               # Verbosity mode
    restore_best_weights=True # Restore model weights from the epoch with the best value of the monitored quantity
)


################# Optionally, an existing model can be loaded in. Below is the code to load in a pre-trained model .#############################
# os.chdir(r"XXXX") # put path to your model here
# model = load_model('ANN4_1000.keras')

# Train the model.
history = model.fit(
    x=x_train, # Input data
    y=y_train, # Output data 
    batch_size=32, # Batch size (adjust as needed)
    epochs=200, # Number of epochs (adjust as needed)
    validation_data=(x_val, y_val), # Validation data
    verbose=1, # Verbosity mode
    callbacks=[early_stopping] # Ends training when performance on validation data stops improving
)

# Change the directory to location to save the model 
os.chdir(r"XXXX")  ########## PLEASE PUT THE DIRECTORY WHERE YOU WOULD LIKE TO SAVE THE NEW MODEL in the XXXX space ####################

##################### To save the model use the code below: ################################
model.save_weights('ANN4_1000.weights.h5') #### Filename is ANN4_(# of training files)
model.save('ANN4_1000.keras')


The next model will be trained with a training set size of 10000

In [ ]:
# Get training data size = 10000

# Identify the location of the data 
path = r'XXXX'  ########## PLEASE PUT THE DIRECTORY OF THE TRAINING DATA OF DATA SET 2 with 10000 files in the XXXX space ####################

# Get the data using function "get_data_aSi_CL_E"
train_files, train_data, train_Ep, train_Eg, train_Eo, train_Br, train_Amp, train_Einf, train_BulkT, train_EMA_bool, train_EMAT, train_Sub_bool =  get_data_CL(path)

# Randomly Shuffle the data using function "unison_shuffled_copies"
train_files, train_data, train_Ep, train_Eg, train_Eo, train_Br, train_Amp, train_Einf, train_BulkT, train_EMA_bool, train_EMAT, train_Sub_bool  = unison_shuffled_copies( train_files, train_data, train_Ep, train_Eg, train_Eo, train_Br, train_Amp, train_Einf, train_BulkT, train_EMA_bool, train_EMAT, train_Sub_bool )


In [ ]:
# Get validation data size = 1000

# Identify the location of the data 
path = r'XXXX'  ########## PLEASE PUT THE DIRECTORY OF THE VALIDATION DATA OF DATA SET 2 with 1000 files in the XXXX space ####################

# Get the data using function "get_data_aSi_CL_E"
val_files, val_data, val_Ep, val_Eg, val_Eo, val_Br, val_Amp, val_Einf, val_BulkT, val_EMA_bool, val_EMAT, val_Sub_bool =  get_data_CL(path)

# Randomly Shuffle the data using function "unison_shuffled_copies"
val_files, val_data, val_Ep, val_Eg, val_Eo, val_Br, val_Amp, val_Einf, val_BulkT, val_EMA_bool, val_EMAT, val_Sub_bool  = unison_shuffled_copies( val_files, val_data, val_Ep, val_Eg, val_Eo, val_Br, val_Amp, val_Einf, val_BulkT, val_EMA_bool, val_EMAT, val_Sub_bool )

In [ ]:
# Use the new function:
path = r'XXXX'  ########## PLEASE PUT THE DIRECTORY WHERE YOU WOULD LIKE TO STORE THE MEAN AND STANDARD DEVIATIONS ####################
filename = 'Data_Set2_10000'

# Get Standardized data
x_train, y_train, x_val, y_val = Standardize_data_ANN4(path, filename)

In [ ]:
# Define input shape
input_shape = (697, 4)
input_NCS = tf.keras.Input(shape=input_shape)

# Flatten the input
x = tfl.Flatten()(input_NCS)

# Hidden Layers
x = tfl.Dense(units=2048, activation='leaky_relu')(x)
x = tfl.Dense(units=1024, activation='leaky_relu')(x)
x = tfl.Dense(units=512, activation='leaky_relu')(x)
x = tfl.Dense(units=256, activation='leaky_relu')(x)
x = tfl.Dense(units=128, activation='leaky_relu')(x)
x = tfl.Dense(units=64, activation='leaky_relu')(x)
x = tfl.Dense(units=32, activation='leaky_relu')(x)

# Output layer has 7 units. These units corrispond to [ Ep, Eg, Eo, Br, Amp, BulkT, EMAT]
x = tfl.Dense(units=7, activation= None)(x)

# Create the model
model = tf.keras.Model(inputs=input_NCS, outputs= x)

# Define the learning rate. The learning rate will exponetially decay with time as the model learns. This technique sometimes helps models converge.
initial_learning_rate = 1e-6 # The learning rate may need to be adjusted with training size.
decay_steps = 500  # How many batches will be done before a decay step
decay_rate = 0.75   # How much the learning rate decays at each step. 

#This scheduler exponentially decays the learning rate based on the parameters above.
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=initial_learning_rate,
    decay_steps=decay_steps,
    decay_rate=decay_rate,
    staircase=True
)

# Compile the model. 
# The optimizer used is Adam. This is an extremly popular optimizer for training ANNs.
# The loss function is Mean Squared Error. Used for regression tasks
model.compile(
    optimizer=tf.keras.optimizers.Adam(lr_schedule),
    loss =  tf.keras.losses.MeanSquaredError()
)

# Define early stopping. This technique calculates the loss from the validation data after every pass through the training data. 
# It will keep track of the validation loss and stop the training after the validation loss becomes stagnate. This ensures that 
# the model does not overtrain on the training data

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',      # Monitor validation loss
    patience=10,              # Number of epochs with no improvement after which training will be stopped
    min_delta=0.0001, 
    verbose=1,               # Verbosity mode
    restore_best_weights=True # Restore model weights from the epoch with the best value of the monitored quantity
)


################# Optionally, an existing model can be loaded in. Below is the code to load in a pre-trained model .#############################
#os.chdir(r"XXXX") # put path to your model here
#model = load_model('ANN4_10000.keras')


# Train the model.
history = model.fit(
    x=x_train, # Input data
    y=y_train, # Output data 
    batch_size=32, # Batch size (adjust as needed)
    epochs=200, # Number of epochs (adjust as needed)
    validation_data=(x_val, y_val), # Validation data
    verbose=1, # Verbosity mode
    callbacks=[early_stopping] # Ends training when performance on validation data stops improving
)

# Change the directory to location to save the model 
os.chdir(r"XXXX")  ########## PLEASE PUT THE DIRECTORY WHERE YOU WOULD LIKE TO SAVE THE NEW MODEL in the XXXX space ####################

##################### To save the model use the code below: ################################
model.save_weights('ANN4_10000_test.weights.h5') #### Filename is ANN4_(# of training files)
model.save('ANN4_10000_test.keras')

The next model will be trained with a training set size of 50000

In [ ]:
# Get training data size = 50000

# Identify the location of the data 
path = r'XXXX'  ########## PLEASE PUT THE DIRECTORY OF THE TRAINING DATA OF DATA SET 2 with 50000 files in the XXXX space ####################

# Get the data using function "get_data_aSi_CL_E"
train_files, train_data, train_Ep, train_Eg, train_Eo, train_Br, train_Amp, train_Einf, train_BulkT, train_EMA_bool, train_EMAT, train_Sub_bool =  get_data_CL(path)

# Randomly Shuffle the data using function "unison_shuffled_copies"
train_files, train_data, train_Ep, train_Eg, train_Eo, train_Br, train_Amp, train_Einf, train_BulkT, train_EMA_bool, train_EMAT, train_Sub_bool  = unison_shuffled_copies( train_files, train_data, train_Ep, train_Eg, train_Eo, train_Br, train_Amp, train_Einf, train_BulkT, train_EMA_bool, train_EMAT, train_Sub_bool )


In [ ]:
# Get validation data size = 100

# Identify the location of the data 
path = r'XXXX'  ########## PLEASE PUT THE DIRECTORY OF THE VALIDATION DATA OF DATA SET 2 with 5000 files in the XXXX space ####################

# Get the data using function "get_data_aSi_CL_E"
val_files, val_data, val_Ep, val_Eg, val_Eo, val_Br, val_Amp, val_Einf, val_BulkT, val_EMA_bool, val_EMAT, val_Sub_bool =  get_data_CL(path)

# Randomly Shuffle the data using function "unison_shuffled_copies"
val_files, val_data, val_Ep, val_Eg, val_Eo, val_Br, val_Amp, val_Einf, val_BulkT, val_EMA_bool, val_EMAT, val_Sub_bool  = unison_shuffled_copies( val_files, val_data, val_Ep, val_Eg, val_Eo, val_Br, val_Amp, val_Einf, val_BulkT, val_EMA_bool, val_EMAT, val_Sub_bool )

In [ ]:
# Use the new function:
path = r'XXXX'   ########## PLEASE PUT THE DIRECTORY WHERE YOU WOULD LIKE TO STORE THE MEAN AND STANDARD DEVIATIONS ####################
filename = 'Data_Set2_50000'

# Get Standardized data
x_train, y_train, x_val, y_val = Standardize_data_ANN4(path, filename)

In [ ]:
# Define input shape
input_shape = (697, 4)
input_NCS = tf.keras.Input(shape=input_shape)

# Flatten the input
x = tfl.Flatten()(input_NCS)

# Hidden Layers
x = tfl.Dense(units=2048, activation='leaky_relu')(x)
x = tfl.Dense(units=1024, activation='leaky_relu')(x)
x = tfl.Dense(units=512, activation='leaky_relu')(x)
x = tfl.Dense(units=256, activation='leaky_relu')(x)
x = tfl.Dense(units=128, activation='leaky_relu')(x)
x = tfl.Dense(units=64, activation='leaky_relu')(x)
x = tfl.Dense(units=32, activation='leaky_relu')(x)

# Output layer has 7 units. These units corrispond to [ Ep, Eg, Eo, Br, Amp, BulkT, EMAT]
x = tfl.Dense(units=7, activation= None)(x)

# Create the model
model = tf.keras.Model(inputs=input_NCS, outputs= x)

# Define the learning rate. The learning rate will exponetially decay with time as the model learns. This technique sometimes helps models converge.
initial_learning_rate = 1e-3 # The learning rate may need to be adjusted with training size.
decay_steps = 2500  # How many batches will be done before a decay step
decay_rate = 0.75   # How much the learning rate decays at each step. 

#This scheduler exponentially decays the learning rate based on the parameters above.
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=initial_learning_rate,
    decay_steps=decay_steps,
    decay_rate=decay_rate,
    staircase=True
)

# Compile the model. 
# The optimizer used is Adam. This is an extremly popular optimizer for training ANNs.
# The loss function is Mean Squared Error. Used for regression tasks
model.compile(
    optimizer=tf.keras.optimizers.Adam(lr_schedule),
    loss =  tf.keras.losses.MeanSquaredError()
)

# Define early stopping. This technique calculates the loss from the validation data after every pass through the training data. 
# It will keep track of the validation loss and stop the training after the validation loss becomes stagnate. This ensures that 
# the model does not overtrain on the training data

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',      # Monitor validation loss
    patience=10,              # Number of epochs with no improvement after which training will be stopped
    #min_delta=0.00001, 
    verbose=1,               # Verbosity mode
    restore_best_weights=True # Restore model weights from the epoch with the best value of the monitored quantity
)


################# Optionally, an existing model can be loaded in. Below is the code to load in a pre-trained model .#############################
#os.chdir(r"C:\Users\bordo\Documents\UToledo\Research\ML\a-Si-Paper\Cody-Lorentz_Reviewer_Comments\Models") # put path to your model here
#model = load_model('ANN4_50000.keras')


# Train the model.
history = model.fit(
    x=x_train, # Input data
    y=y_train, # Output data 
    batch_size=32, # Batch size (adjust as needed)
    epochs=200, # Number of epochs (adjust as needed)
    validation_data=(x_val, y_val), # Validation data
    verbose=1, # Verbosity mode
    callbacks=[early_stopping] # Ends training when performance on validation data stops improving
)

# Change the directory to location to save the model 
os.chdir(r"XXXX")  ########## PLEASE PUT THE DIRECTORY WHERE YOU WOULD LIKE TO SAVE THE NEW MODEL in the XXXX space ####################

##################### To save the model use the code below: ################################
model.save_weights('ANN4_50000v2.weights.h5') #### Filename is ANN4_(# of training files)
model.save('ANN4_50000v2.keras')


In [ ]:

# Train the model.
history = model.fit(
    x=x_train, # Input data
    y=y_train, # Output data 
    batch_size=32, # Batch size (adjust as needed)
    epochs=5, # Number of epochs (adjust as needed)
    validation_data=(x_val, y_val), # Validation data
    verbose=1, # Verbosity mode
    callbacks=[early_stopping] # Ends training when performance on validation data stops improving
)

# Change the directory to location to save the model 
os.chdir(r"XXXX")  ########## PLEASE PUT THE DIRECTORY WHERE YOU WOULD LIKE TO SAVE THE NEW MODEL in the XXXX space ####################

##################### To save the model use the code below: ################################
model.save_weights('ANN4_50000v2.weights.h5') #### Filename is ANN4_(# of training files)
model.save('ANN4_50000v2.keras')


The next model will be trained with a training set size of 100000

In [ ]:
# Get training data size = 100000

# Identify the location of the data 
path = r'XXXX'  ########## PLEASE PUT THE DIRECTORY OF THE TRAINING DATA OF DATA SET 2 with 100000 files in the XXXX space ####################

# Get the data using function "get_data_aSi_CL_E"
train_files, train_data, train_Ep, train_Eg, train_Eo, train_Br, train_Amp, train_Einf, train_BulkT, train_EMA_bool, train_EMAT, train_Sub_bool =  get_data_CL(path)

# Randomly Shuffle the data using function "unison_shuffled_copies"
train_files, train_data, train_Ep, train_Eg, train_Eo, train_Br, train_Amp, train_Einf, train_BulkT, train_EMA_bool, train_EMAT, train_Sub_bool  = unison_shuffled_copies( train_files, train_data, train_Ep, train_Eg, train_Eo, train_Br, train_Amp, train_Einf, train_BulkT, train_EMA_bool, train_EMAT, train_Sub_bool )


In [ ]:
# Get validation data size = 10000

# Identify the location of the data 
path = r'XXXX'  ########## PLEASE PUT THE DIRECTORY OF THE VALIDATION DATA OF DATA SET 2 with 10000 files in the XXXX space ####################

# Get the data using function "get_data_aSi_CL_E"
val_files, val_data, val_Ep, val_Eg, val_Eo, val_Br, val_Amp, val_Einf, val_BulkT, val_EMA_bool, val_EMAT, val_Sub_bool =  get_data_CL(path)

# Randomly Shuffle the data using function "unison_shuffled_copies"
val_files, val_data, val_Ep, val_Eg, val_Eo, val_Br, val_Amp, val_Einf, val_BulkT, val_EMA_bool, val_EMAT, val_Sub_bool  = unison_shuffled_copies( val_files, val_data, val_Ep, val_Eg, val_Eo, val_Br, val_Amp, val_Einf, val_BulkT, val_EMA_bool, val_EMAT, val_Sub_bool )

In [ ]:
# Use the new function:
path = r'XXXX' ########## PLEASE PUT THE DIRECTORY WHERE YOU WOULD LIKE TO STORE THE MEAN AND STANDARD DEVIATIONS ####################
filename = 'Data_Set2_100000'

# Get Standardized data
x_train, y_train, x_val, y_val = Standardize_data_ANN4(path, filename)

In [ ]:
# Define input shape
input_shape = (697, 4)
input_NCS = tf.keras.Input(shape=input_shape)

# Flatten the input
x = tfl.Flatten()(input_NCS)

# Hidden Layers
x = tfl.Dense(units=2048, activation='leaky_relu')(x)
x = tfl.Dense(units=1024, activation='leaky_relu')(x)
x = tfl.Dense(units=512, activation='leaky_relu')(x)
x = tfl.Dense(units=256, activation='leaky_relu')(x)
x = tfl.Dense(units=128, activation='leaky_relu')(x)
x = tfl.Dense(units=64, activation='leaky_relu')(x)
x = tfl.Dense(units=32, activation='leaky_relu')(x)

# Output layer has 7 units. These units corrispond to [ Ep, Eg, Eo, Br, Amp, BulkT, EMAT]
x = tfl.Dense(units=7, activation= None)(x)

# Create the model
model = tf.keras.Model(inputs=input_NCS, outputs= x)

# Define the learning rate. The learning rate will exponetially decay with time as the model learns. This technique sometimes helps models converge.
initial_learning_rate = 1e-5 # The learning rate may need to be adjusted with training size.
decay_steps = 5000  # How many batches will be done before a decay step
decay_rate = 0.75   # How much the learning rate decays at each step. 

#This scheduler exponentially decays the learning rate based on the parameters above.
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=initial_learning_rate,
    decay_steps=decay_steps,
    decay_rate=decay_rate,
    staircase=True
)

# Compile the model. 
# The optimizer used is Adam. This is an extremly popular optimizer for training ANNs.
# The loss function is Mean Squared Error. Used for regression tasks
model.compile(
    optimizer=tf.keras.optimizers.Adam(lr_schedule),
    loss =  tf.keras.losses.MeanSquaredError()
)

# Define early stopping. This technique calculates the loss from the validation data after every pass through the training data. 
# It will keep track of the validation loss and stop the training after the validation loss becomes stagnate. This ensures that 
# the model does not overtrain on the training data

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',      # Monitor validation loss
    patience=10,              # Number of epochs with no improvement after which training will be stopped
    min_delta=0.0001, 
    verbose=1,               # Verbosity mode
    restore_best_weights=True # Restore model weights from the epoch with the best value of the monitored quantity
)


################# Optionally, an existing model can be loaded in. Below is the code to load in a pre-trained model .#############################
os.chdir(r"XXXX") # put path to your model here
model = load_model('ANN4_100000.keras')


# Train the model.
history = model.fit(
    x=x_train, # Input data
    y=y_train, # Output data 
    batch_size=32, # Batch size (adjust as needed)
    epochs=200, # Number of epochs (adjust as needed)
    validation_data=(x_val, y_val), # Validation data
    verbose=1, # Verbosity mode
    callbacks=[early_stopping] # Ends training when performance on validation data stops improving
)

# Change the directory to location to save the model 
os.chdir(r"XXXX")  ########## PLEASE PUT THE DIRECTORY WHERE YOU WOULD LIKE TO SAVE THE NEW MODEL in the XXXX space ####################

##################### To save the model use the code below: ################################
model.save_weights('ANN4_100000.weights.h5') #### Filename is ANN4_(# of training files)
model.save('ANN4_100000.keras')


All of the ANNs have been trained. Their perfromance will be evaluated in the next workbook.